# The Rosendrock method

Usually in real-world-problems one encounters a system of differential equations, which does not behave nicely. Such systems are called *stiff*, and basically it means that it becomes expensive for an adaptive method to find a small enough stepsize so that the solution does not explode. 

As far aas know implicit methods are best, i.e. methods where one has to solve an equation of the form
$$ F(\vec{k}_1 ,\vec{k}_2,\vec{k}_3 \cdots ) = G(\vec{k}_1 ,\vec{k}_2,\vec{k}_3 \cdots ),$$
which also becomes expensive and difficult to implement. 


However, there is one method that can give nice results fairly cheaply, and is called the Rosendrock method.

Lets assume, as usual, that we have a differential equation
$$
\dfrac{d\vec{y}}{dt}=\vec{f}(\vec{y},t) \;,
$$
with  given $\vec{y}(0)$ (as always $t \in [0,1]$). In quite involveed methods, in order to simplify 
the notation, we take $t$ as a component of $\vec{y}$. So if we have $N$ differential equations with
explicit t-dependence, we now have $N+1$, for which $y_{N+1}=t$ with $\dfrac{dy_{N+1}}{dt}=1$ and $t_0 =0$.
Doing so, we can follow the RK algorithms, and  $t_{n+1}=t_{n}+h$, $\sum_i b_i =1$, $c_i \sum_i a_{ji}$, 
we find that $k_{N+1,i}=h$. 


So it suffices to study the solution of

$$
\dfrac{d\vec{y}}{dt}=\vec{f}(\vec{y}) \;,
$$
i.e remove the explicit t dependence. We follow then the standard iteration:

$$
\vec{y}_{n+1}=\vec{y}_{n}+ \sum_{i=1}^{s} b_i \vec{k}_i \\
\vec{y}^{\star}_{n+1}=\vec{y}_{n}+ \sum_{i=1}^{s} b_i^{\star} \vec{k}_i \;.
$$

The Rosenbrock method says that the $\vec{k}$'s are given by

$$
\left(\hat I - \gamma_i h \hat J \right)\cdot \vec{k}_i =  
h \vec{f}\Big(\vec{y}_n+\sum_{j=1}^{i-1}a_{ij}\vec{k}_{j} \Big)+
h \hat J \cdot \sum_{j=1}^{i-1}\gamma_{ij}\vec{k}_{j},
$$
where the  $\alpha, b, c$, and $\gamma$ parameters define the method, $\hat I$ is the $N+1,N+1$ unit matrix,
and $\hat J =\dfrac{\partial \vec{f}}{\partial \vec{y}}_{\vec{y}=\vec{y}_n} $ is the Jacobian of the system.
Particularly interested methods include those with $\gamma_i=\gamma \; \forall i$, since we can just calculate the LU-factorisation of $\left(\hat I - \gamma h \hat J \right)$ once in every step. This is the case in which I'm focusing here.

To recap, I'll make a class that will solve differential equations using 

$$
\vec{y}_{n+1}=\vec{y}_{n}+ \sum_{i=1}^{s} b_i \vec{k}_i \\
\vec{y}^{\star}_{n+1}=\vec{y}_{n}+ \sum_{i=1}^{s} b_i^{\star} \vec{k}_i  \\
\left(\hat I - \gamma h \hat J \right)\cdot \vec{k}_i =  
h \vec{f}\Big(\vec{y}_n+\sum_{j=1}^{i-1}a_{ij}\vec{k}_{j} \Big)+
h \hat J \cdot \sum_{j=1}^{i-1}\gamma_{ij}\vec{k}_{j} \;.
$$


To do this I will have to discuss how to actually implement the LU method for solving linear systems of equations. I have an LU solver function ready but not documented, so I'll just use the scipy one.

In [1]:
#I will make the jacobian passed as input in ROS.
class Jacobian:
    def __init__(self,diffeq,h=1e-5):
        self.n=diffeq.n_eqs+1 #+1 to account for time
        self.dydt=diffeq
        self.h=h
        
    
                

    def __call__(self,y,t,J):#J is passed by reference
        
        
        dydt0=self.dydt(y,t-self.h) 
        dydt1=self.dydt(y,t+self.h) 
        for i in range(self.n-1):
            J[i][self.n-1]=(dydt1[i]-dydt0[i])/(2*self.h)
        J[self.n-1][self.n-1]=1.
        for i in range(self.n-1):
            for j in range(self.n-1):
                #all y's are constant, except theit j component
                y0=y[:]
                y1=y[:]
                y0[j]=y[j]-self.h
                y1[j]=y[j]+self.h
                
                dydt0=self.dydt(y0,t) 
                dydt1=self.dydt(y1,t) 
                
                J[i][j]=(dydt1[i]-dydt0[i])/(2*self.h)
        return J

In [4]:
class diff_eq:
    def __init__(self,n=1):
        self.n_eqs=n
        
    
    def __call__(self,y,t):
        return [y[0]*t,y[1]*t,y[2]*y[0],t ]
        
    

    
dydt=diff_eq(4)

j=Jacobian(dydt)

J=[ [0 for j in range(5)] for i in range(5)]
y=[5,10,3,1.55]
j(y,1.55,J)

[[1.549999999950202, 0.0, 0.0, 0.0, 5.000000000032756],
 [0.0, 1.5499999999057932, 0.0, 0.0, 10.000000000065512],
 [2.9999999998864264, 0.0, 4.999999999988347, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 1.0000000000065512],
 [0, 0, 0, 0, 1.0]]

In [57]:
#for LU 
from scipy.linalg import lu_factor as LU
from scipy.linalg import lu_solve as LU_solve




import numpy as np

import matplotlib
#matplotlib.use('WebAgg')
#matplotlib.use('Qt4Cairo')
#matplotlib.use('Qt5Cairo')
matplotlib.use('nbAgg')
import matplotlib.pyplot as plt

plt.rcParams['font.family']='serif'
plt.rcParams['font.size']=10
plt.rcParams['mathtext.fontset']='stixsans'

In [58]:
#I'll use ROS3w here, since I've the parameters ready.
class ROS3w:
    def __init__(self):
        #third stage, second order method
        self.s=3
        self.p=2
        self.c=[0,2/3.,4/3.]
        self.b=[0.25,0.25,0.5  ]
        self.bstar=[0.746704703274 , 0.1144064078371 , 0.1388888888888]
        self.a=[ [0 for j in range(self.s)] for i in range(self.s)]
        self.gamma=0.4358665215084 #this is gamma
        self.g=[ [0 for j in range(self.s)] for i in range(self.s)]#this is gamma_{ij}
        
        self.a[1][0]=2/3.
        self.a[2][0]=2/3.
        self.a[2][1]=2/3.
        
        self.g[1][0]=0.3635068368900
        self.g[2][0]=-0.8996866791992
        self.g[2][1]=-0.1537997822626
        
        